In [ ]:
import re
import string
import numpy as np
import pandas as pd
from collections import Counter


In [2]:
COMMENT = 'comment_text'
LABELS = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
SENTIMENT = ['polarity', 'subjectivity']

In [3]:
train = pd.read_csv('jigsaw-toxic-comment-classification-challenge/train.csv')
test = pd.read_csv('jigsaw-toxic-comment-classification-challenge/test.csv')
test_labels = pd.read_csv('jigsaw-toxic-comment-classification-challenge/test_labels.csv')
sample_submission = pd.read_csv('jigsaw-toxic-comment-classification-challenge/sample_submission.csv')

# Problem understanding

In [4]:
print(train.shape)
print(train.columns)
train.head(3)

(159571, 8)
Index(['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate'],
      dtype='object')


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0


In [5]:
train[LABELS].sum()

toxic            15294
severe_toxic      1595
obscene           8449
threat             478
insult            7877
identity_hate     1405
dtype: int64

In [6]:
train.describe()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
count,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000
mean,0.095844,0.009996,0.052948,0.002996,0.049364,0.008805
std,0.294379,0.099477,0.223931,0.054650,0.216627,0.093420
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [7]:
test.describe()

,id,comment_text
count,153164,153164
unique,153164,153164
top,a538bbade1c5d837,== Her parents are both Sinhalese? == \n\n Acc...
freq,1,1


In [8]:
for k, v in Counter(train[LABELS].sum(axis=1)).items():
    print(k, '{:0.2%}'.format(v/len(train)))

0 89.83%
4 1.10%
1 3.99%
3 2.64%
2 2.18%
5 0.24%
6 0.02%


In [9]:
# -1: not scoring
for k, v in Counter(test_labels[LABELS].sum(axis=1)).items():
    print(k, '{:0.2%}'.format(v/len(test_labels)))

-6 58.23%
0 37.69%
1 1.20%
4 0.40%
3 1.36%
2 1.00%
5 0.11%
6 0.01%


In [10]:
Counter(train[LABELS].values.reshape(-1))

Counter({0: 922328, 1: 35098})

In [11]:
Counter(test_labels[LABELS].values.reshape(-1))

Counter({-1: 535116, 0: 369370, 1: 14498})

In [12]:
for l in LABELS:
    c = Counter(train[l])
    print(l, '{:.2%}'.format(c[1]/sum(c.values())), c)

toxic 9.58% Counter({0: 144277, 1: 15294})
severe_toxic 1.00% Counter({0: 157976, 1: 1595})
obscene 5.29% Counter({0: 151122, 1: 8449})
threat 0.30% Counter({0: 159093, 1: 478})
insult 4.94% Counter({0: 151694, 1: 7877})
identity_hate 0.88% Counter({0: 158166, 1: 1405})


In [13]:
for l in LABELS:
    c = Counter(train[train[l] == 1]['toxic'])
    print(l, '{:.2%}'.format(c[1]/sum(c.values())), c)

toxic 100.00% Counter({1: 15294})
severe_toxic 100.00% Counter({1: 1595})
obscene 93.81% Counter({1: 7926, 0: 523})
threat 93.93% Counter({1: 449, 0: 29})
insult 93.23% Counter({1: 7344, 0: 533})
identity_hate 92.67% Counter({1: 1302, 0: 103})


In [14]:
for l in LABELS:
    c = Counter(train[train['toxic'] == 1][l])
    print(l, '{:.2%}'.format(c[1]/sum(c.values())), c)

toxic 100.00% Counter({1: 15294})
severe_toxic 10.43% Counter({0: 13699, 1: 1595})
obscene 51.82% Counter({1: 7926, 0: 7368})
threat 2.94% Counter({0: 14845, 1: 449})
insult 48.02% Counter({0: 7950, 1: 7344})
identity_hate 8.51% Counter({0: 13992, 1: 1302})


In [15]:
train[LABELS].corr()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
toxic,1.000000,0.308619,0.676515,0.157058,0.647518,0.266009
severe_toxic,0.308619,1.000000,0.403014,0.123601,0.375807,0.201600
obscene,0.676515,0.403014,1.000000,0.141179,0.741272,0.286867
threat,0.157058,0.123601,0.141179,1.000000,0.150022,0.115128
insult,0.647518,0.375807,0.741272,0.150022,1.000000,0.337736
identity_hate,0.266009,0.201600,0.286867,0.115128,0.337736,1.000000


In [16]:
for i in LABELS:
    for j in LABELS:
        c = Counter(train[train[j] == 1][i])
        print('[%{} when {}==1] {:.2%}'.format(i, j, c[1]/sum(c.values())))

[%toxic when toxic==1] 100.00%
[%toxic when severe_toxic==1] 100.00%
[%toxic when obscene==1] 93.81%
[%toxic when threat==1] 93.93%
[%toxic when insult==1] 93.23%
[%toxic when identity_hate==1] 92.67%
[%severe_toxic when toxic==1] 10.43%
[%severe_toxic when severe_toxic==1] 100.00%
[%severe_toxic when obscene==1] 17.95%
[%severe_toxic when threat==1] 23.43%
[%severe_toxic when insult==1] 17.41%
[%severe_toxic when identity_hate==1] 22.28%
[%obscene when toxic==1] 51.82%
[%obscene when severe_toxic==1] 95.11%
[%obscene when obscene==1] 100.00%
[%obscene when threat==1] 62.97%
[%obscene when insult==1] 78.14%
[%obscene when identity_hate==1] 73.45%
[%threat when toxic==1] 2.94%
[%threat when severe_toxic==1] 7.02%
[%threat when obscene==1] 3.56%
[%threat when threat==1] 100.00%
[%threat when insult==1] 3.90%
[%threat when identity_hate==1] 6.98%
[%insult when toxic==1] 48.02%
[%insult when severe_toxic==1] 85.96%
[%insult when obscene==1] 72.85%
[%insult when threat==1] 64.23%
[%insult w

In [17]:
print(test.shape)
print(test.columns)
test.head(3)

(153164, 2)
Index(['id', 'comment_text'], dtype='object')


,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."


In [18]:
print(test_labels.shape)
print(test_labels.columns)
test_labels.head(3)

(153164, 7)
Index(['id', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult',
       'identity_hate'],
      dtype='object')


,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,-1,-1,-1,-1,-1,-1
1,0000247867823ef7,-1,-1,-1,-1,-1,-1
2,00013b17ad220c46,-1,-1,-1,-1,-1,-1


In [19]:
print(sample_submission.shape)
print(sample_submission.columns)
sample_submission.head(3)

(153164, 7)
Index(['id', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult',
       'identity_hate'],
      dtype='object')


,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.5,0.5,0.5,0.5,0.5,0.5
1,0000247867823ef7,0.5,0.5,0.5,0.5,0.5,0.5
2,00013b17ad220c46,0.5,0.5,0.5,0.5,0.5,0.5


# Dataset generation

# Metrics define

# Validation strategy

# Data processing (extract useful information)

# Data understanding & visualization

# De-noise (no drop data)

# Feature engineering

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer

vec = TfidfVectorizer(max_features=16384)

x_train = vec.fit_transform(train[COMMENT])
x_test = vec.transform(test[COMMENT])

# Offline augmentation

# Standarization

# Scaling

# Normalization

# Feature selection

# Data selection

# Optimization

In [21]:
from sklearn.linear_model import LogisticRegression
models = {l: LogisticRegression() for l in LABELS}
[models[l].fit(x_train, train[l]) for l in LABELS]
for l in LABELS:
    sample_submission[l] = models[l].predict_proba(x_test)[:, 1]
sample_submission.to_csv('1.submission.csv', index=False)

# ~ 97.42%

/home/shanger_lin/.pyenv/versions/3.6.4/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [23]:
for l in LABELS:
    sample_submission[l] = models[l].predict_proba(x_test)[:, 1]
sample_submission.loc[:, 'severe_toxic'][sample_submission['toxic'] < 0.2] = 0
sample_submission.to_csv('2.submission.csv', index=False)
# ~ 97.46%

/home/shanger_lin/.pyenv/versions/3.6.4/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [24]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.regularizers import l2

keras_input = keras.Input(shape=[x_test.shape[1]])
l = layers.Dense(1024, activation='relu', kernel_regularizer=l2(1e-5))(keras_input)
l = layers.Dense(512, activation='relu', kernel_regularizer=l2(1e-5))(l)
l = layers.Dense(128, activation='relu', kernel_regularizer=l2(1e-5))(l)
keras_output = layers.Dense(6, activation='sigmoid')(l)
model = keras.Model(keras_input, keras_output)
model.compile(keras.optimizers.Adam(3e-4), loss='binary_crossentropy', metrics=['acc'])

model.fit(x_train.todense(), 
          train[LABELS], 
          validation_split=0.05,
          epochs=10, 
          batch_size=128,
          shuffle=True, 
          callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=1, verbose=1), ])

y_pred = model.predict(x_test.todense())

for i, l in enumerate(LABELS):
    sample_submission[l] = y_pred[:, i]
sample_submission.to_csv('3.submission.csv', index=False)
# ~ 97.37%

KeyboardInterrupt: 

In [ ]:
from sklearn.ensemble import RandomForestRegressor
models = {l: RandomForestRegressor(n_estimators=1, n_jobs=-1, max_depth=100) for l in LABELS}

for l in LABELS:
    print(l)
    models[l].fit(x_train, train[l])
    sample_submission[l] = models[l].predict(x_test)
sample_submission.to_csv('4.submission.csv', index=False)

# ~ 58.21% (might dure to max_depth and n_estimators limit for training time constrain)

In [25]:
models = {l: LogisticRegression(class_weight='balanced') for l in LABELS}
for l in LABELS:
    models[l].fit(x_train, train[l])
    sample_submission[l] = models[l].predict_proba(x_test)[:, 1]
sample_submission.to_csv('5.submission.csv', index=False)

# ~ 97.49%

/home/shanger_lin/.pyenv/versions/3.6.4/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [26]:
model = sklearn.ensemble.ExtraTreesClassifier(n_jobs=-1, n_estimators=10)
model.fit(x_train, train[LABELS])
y_pred = model.predict_proba(x_test)[0][:, 1]
for i, l in enumerate(LABELS):
    sample_submission[l] = y_pred
sample_submission.to_csv('6.submission.csv', index=False)
# ~ 94.15%

NameError: name 'sklearn' is not defined

In [ ]:
model = sklearn.ensemble.ExtraTreesClassifier(n_jobs=-1, n_estimators=1000)
model.fit(x_train, train[LABELS])
y_pred = model.predict_proba(x_test)[0][:, 1]
for i, l in enumerate(LABELS):
    sample_submission[l] = y_pred
sample_submission.to_csv('7.submission.csv', index=False)
# ~ 95.66%

# Parameter tuning

# Online augmentation

# Model selection / blending

# Post-processing

# Evaluation

# Reasoning

# Monitoring